In [1]:
import io,os,sys,json
from types import SimpleNamespace

notebook_dir = os.getcwd()
print('notebook dir: ', notebook_dir)
project_dir = os.path.dirname(notebook_dir)
print('project dir: ', project_dir)
sys.path.insert(0, project_dir)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dingdong.settings")

import django
django.setup()

def print_qs(querySet):
    from django.core import serializers
    data = serializers.serialize("json", querySet, indent=4)
    print(data)

notebook dir:  D:\workspace\python\dingdong\notebooks
project dir:  D:\workspace\python\dingdong


In [2]:
from django.contrib.auth import authenticate
from django.contrib.contenttypes.models import ContentType

In [3]:
from rest_framework.request import Request
from rest_framework.test import APIRequestFactory

request_factory = APIRequestFactory()

serializer_context = {
    'request': Request(request_factory.get('/')),
}

from rest_framework.test import RequestsClient
from rest_framework import serializers
from rest_framework.renderers import JSONRenderer

In [4]:
from base.models import Account
from base.api.serializers import AccountSerializer
from rest_framework.decorators import action
from rest_framework import viewsets
from rest_framework.response import Response

In [13]:
auth_data = {'username':'admin', 'password':'123'}
##############################################################################################
# authencate
client = RequestsClient()
response = client.post('http://testserver/api/token/obtain/', auth_data)
assert response.status_code == 200
token = json.loads(response.content.decode("utf-8"))
client.headers.update({'Authorization': 'Bearer ' + token['access']})

def get(url):
    response = client.get(url)
#     assert response.status_code == 200
    print('response status code:', response.status_code)
    result = json.loads(response.content.decode("utf-8"))
    print(json.dumps(result, indent=4))
###########################################################################################

In [14]:
class AccountViewSet(viewsets.ModelViewSet):
    """
    API endpoint that allows users to be viewed or edited.
    """
    # permission_classes = (permissions.IsAuthenticatedOrReadOnly,)
    queryset = Account.objects.all().order_by('-date_joined')
    serializer_class = AccountSerializer

    @action(detail=True, name='get all permissions')
    def permissions(self, request, pk=None):
        perms = self.get_object().get_all_permissions()
        return Response(perms)


In [19]:
url = 'http://testserver/api/base/accounts/'
url = 'http://testserver/api/base/accounts/2'
url = 'http://testserver/api/base/accounts/2/permissions'

get(url)

response status code: 200
[
    "base.view_customer",
    "base.view_staff",
    "base.delete_article",
    "base.delete_articlecategory",
    "base.add_articlecategory",
    "base.add_staff",
    "base.change_customer",
    "base.delete_company",
    "base.add_article"
]
